nb. the following code won't run - is just to give example

## Track progress better & save weights after each epoch

I found Jupyter kept freezing while I was running model, and couldn't tell whether it was working or how much progress it had made. Therefore:

1. Follow suggestions to use package to more clearly track progress that can help stop Jupyter freezing. For details of TQDM progress bars see [github](https://github.com/bstriner/keras-tqdm). [This post](http://forums.fast.ai/t/jupyter-notebook-dies-freezes-during-training/2651/4) recommends setting verbose=0 when you do this.

2. Save model after each epoch, so if something goes wrong I dont lose work and so I can get an idea of progress by seeing what files have been saved. For details of saving model after each epoch see [documentation](https://keras.io/callbacks/#modelcheckpoint). There is an example of applying it [here](https://machinelearningmastery.com/check-point-deep-learning-models-keras/). There is an example of how to then load waits and pick up where you left off [here](https://github.com/keras-team/keras/issues/2378) and a discussion [here](https://github.com/keras-team/keras/issues/1872).

In [ ]:
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()
model.add(layers.Flatten(input_shape=(lookback / step, float_data.shape[-1])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

filepath='saved_epochs/stacked-weights-{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False)
callbacks_list = [TQDMNotebookCallback(), checkpoint]

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=5,
                              epochs=1,    # 5
                              validation_data=val_gen,
                              validation_steps=val_steps,
                              callbacks=callbacks_list)

If then need to read in model and pick up where you left off:

In [ ]:
model = Sequential()
model.add(layers.Flatten(input_shape=(lookback / step, float_data.shape[-1])))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
model.load_weights('saved_epochs/stacked-weights-01-89.09.hdf5')

history = model.fit_generator(train_gen,
                              steps_per_epoch=5,
                              epochs=1,    # 5
                              validation_data=val_gen,
                              validation_steps=val_steps)

## Save history

In [ ]:
# save history as a dictionary
def save_obj(obj, name):
    with open('histories/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(history.history, 'model_rnn_history')

In [ ]:
# if need to load dictionary containing history
# (will need to refined loss and val_loss - see the two lines below)
def load_obj(name ):
    with open('histories/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
model_hist = load_obj('model_rnn_history')   

## Save model when done fitting

I think that this will not overwrite an existing file - so be sure to manually delete it first.

In [ ]:
# save model (if it performs best, I will want to evaluate on test set,
# and in practice would want to use it's predictions)
# serialize model to JSON
model_json = model.to_json()
with open("model_basic.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_basic.h5")

Load model:

In [ ]:
# load json and create model
json_file = open('model_basic.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_basic.h5")
print("Loaded model from disk")

# compile
loaded_model.compile(loss='mae', optimizer='rmsprop', metrics=['accuracy'])